# Modelo de predição de mortes por dengue em pessoas internadas no Brasil

## Agenda

### Dia 01 - Treinamendo do modelo
- 01. Importação de dados com o PySUS
- 02. Tratamento dos dados
- 03. Análise de correlação e associação das variáveis
- 04. Treinamento do modelo

### Dia 2 - Alternativas para aprimorar o modelo
- 05. Alterando o threshold do modleo
- 06. Trabalhando com o desbalanceamento da base

## 00 - Importando as bibliotecas que serão usadas na análise

In [ ]:
#IMPORTANDO PACOTES
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd  
import pylab as pl
import chart_studio.plotly as py
import plotly.graph_objs as gp 
import rpy2.robjects as robjects
import seaborn as sns
import statsmodels.formula.api as smf

from ftplib import FTP
from pysus.online_data.SIH import download
from scipy.stats import chi2, chi2_contingency, iqr, pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support, classification_report
from sklearn.model_selection import train_test_split

## 01 - Importando os dados

In [ ]:
#DEFININDO VARIÁVEIS QUE SERÃO UTILIZADAS NO DOWNLOAD DO PYSUS
vars =["N_AIH","IDADE","SEXO","UF_ZI","NUM_FILHOS","INSTRU","MORTE","CID_MORTE","DIAS_PERM","DIAG_PRINC","DIAG_SECUN","DT_INTER","ANO_CMPT","MES_CMPT","MARCA_UTI","UTI_MES_TO","INFEHOSP"]
ufs = ['ac', 'al', 'ap', 'am','ba', 'ce', 'df', 'es', 'go', 'ma', 'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi', 'rj', 'rn', 'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to']
anos = [2019, 2020, 2021] 
meses = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
cids = ["A90", "A900"]

In [ ]:
#IMPORTANDO DADOS
for mes in meses: 
    for uf in ufs:
        for ano in anos:   
            df = download(uf, ano, mes)
            df = df.filter(vars)
            df = df[df['DIAG_PRINC'].isin(cids)]
            df.to_csv("sih_dengue1.csv", mode='a', index=False, header=False)
            print(f"O arquivo do mês {mes} de {ano} do estado {uf.upper()} foi filtrado")

In [ ]:
#IMPORTANDO OS DADOS DO CSV SALVO
colunas =  ["N_AIH","IDADE","SEXO","UF_ZI","NUM_FILHOS","INSTRU","MORTE","CID_MORTE","DIAS_PERM","DIAG_PRINC","DIAG_SECUN","DT_INTER","ANO_CMPT","MES_CMPT","MARCA_UTI","UTI_MES_TO","INFEHOSP"]

sih_dengue = pd.read_csv("sih_dengue.csv")
sih_dengue.set_axis(colunas, axis= 1, inplace= True)

## 02 - Tratamento dos dados

In [ ]:
#VISUALIZANDO OS DADOS
sih_dengue.head(20)

### 02.1 - Variáveis que não serão usadas no modelo

In [ ]:
#EXCLUINDO VARIÁVEIS QUE NÃO SERÃO USADAS NO MODELO
sih_dengue.drop(["MES_CMPT", "ANO_CMPT", "DT_INTER", "DIAG_PRINC"], axis = 1, inplace = True)

### 02.2 - Missing values

In [ ]:
#VISUALIZANDO OS TIPOS DE DADOS - IDENTIFICANDO MISSING VALUES
sih_dengue.info()

In [ ]:
#IDENTIFICANDO VARIÁVEIS QUE NÃO SERÃO USADAS POR NÃO POSSUÍREM NENHUM VALOR (MISSING VALUES)
sih_dengue["NUM_FILHOS"].unique(), sih_dengue["INSTRU"].unique(), sih_dengue["CID_MORTE"].unique(), sih_dengue["DIAG_SECUN"].unique(), sih_dengue["INFEHOSP"].unique()

In [ ]:
#EXCLUINDO VARIÁVEIS QUE SÃO FORMADAS APENAS DE MISSING VALUES
sih_dengue.drop(["NUM_FILHOS", "INSTRU", "CID_MORTE", "INFEHOSP", "DIAG_SECUN"], axis = 1, inplace = True)

### 02.3 - Transformando as variáveis

In [ ]:
#TRANSFORMANDO VARIÁVEIS
##número da AIH - transformando em string e atribuindo como index
sih_dengue["N_AIH"] = sih_dengue["N_AIH"].astype(str)
sih_dengue.set_index(['N_AIH'], inplace= True)

##sexo
sexo = {1:"Masculino", 3:"Feminino"}
sih_dengue["SEXO"] = sih_dengue["SEXO"].replace(sexo)

##região
regiao = {"1":"Norte", "2":"Nordeste", "3":"Sudeste", "4":"Sul", "5":"Centro-Oeste"}
sih_dengue["UF_ZI"] = sih_dengue["UF_ZI"].astype(str)
sih_dengue["REGIAO"] = sih_dengue["UF_ZI"].str.slice(0,1)
sih_dengue["REGIAO"] = sih_dengue["REGIAO"].replace(regiao)
sih_dengue.drop(["UF_ZI"], axis = 1, inplace = True)

In [ ]:
#VISUALIZANDO VARIÁVEIS APÓS O TRANSFORMAÇÃO 
sih_dengue.info()

In [ ]:
#QUAANTIFICANDO VALORES DAS VARIÁVEIS SEXO E MARCA UTI
sih_dengue["SEXO"].value_counts(), sih_dengue["REGIAO"].value_counts(), sih_dengue["MARCA_UTI"].value_counts()

In [ ]:
#TRANSFORMANDO VARIÁVEIS EM DUMMIES
##sexo
sexo = {"Masculino":0, "Feminino":1}
sih_dengue["SEXO"] = sih_dengue["SEXO"].replace(sexo)

##região de moradia
sih_dengue["REGIAO"] = np.where(sih_dengue["REGIAO"] == "Sudeste", 1, 0)

##estadia em UTI
sih_dengue["MARCA_UTI"] = np.where(sih_dengue["MARCA_UTI"] == 0, 0, 1)

In [ ]:
#VISUALIZANDO VARIÁVEIS APÓS A TRANSFORMAÇÃO
sih_dengue.info()

### 02.3 - Removendo outliers

In [ ]:
#SELECIONANDO VARIÁVEIS QUE PODEM APRESENTAR OUTLIERS
sih_box = sih_dengue[['IDADE', 'DIAS_PERM', 'UTI_MES_TO']]

In [ ]:
#TRANSFORMANDO DADOS PARA A MESMA ESCALA
scale = StandardScaler()

sih_box2 = pd.DataFrame(scale.fit_transform(sih_box))
sih_box2.set_axis(['IDADE', 'DIAS_PERM', 'UTI_MES_TO'], axis=1, inplace=True)
sih_box2 = pd.DataFrame(sih_box2.stack()).reset_index()
sih_box2.drop('level_0', axis = 1, inplace = True)
sih_box2.rename(columns = {'level_1':'variables', 0:'standard_value'}, inplace = True)

In [ ]:
#GERANDO BOXPLOTS
sns.set(rc={'figure.figsize':(14, 10)})
sns.set_theme(style="whitegrid", palette="viridis")

box = sns.boxplot(y = "variables", x = "standard_value", data = sih_box2)
box.set(xlabel ="Valores padronizados", ylabel = "Variáveis", title ='Boxplot dias de permanência e idade padronizados')

In [ ]:
#SELECIONANDO DADOS COM DISTRIBUIÇÃO MAIOR QUE -1.96 E MENOR QUE 1.96
sih_box = pd.DataFrame(sih_box.stack()).reset_index()
sih_box.drop('N_AIH', axis = 1, inplace = True)
sih_box.rename(columns = {'level_1':'variables_originals', 0:'value'}, inplace = True)

sih_box = pd.concat([sih_box.reset_index(drop=True), sih_box2.reset_index(drop=True)], axis=1)
sih_box = sih_box[(sih_box['standard_value'] >= -1.96) & (sih_box['standard_value'] <= 1.96)]

In [ ]:
#SELECIONANDO VALORES MÍNIMOS E MÁXIMOS 
min_values = pd.DataFrame(sih_box.groupby('variables_originals')['value'].min())
max_values = pd.DataFrame(sih_box.groupby('variables_originals')['value'].max())

min_values, max_values

In [ ]:
#GERANDO DATAFRAME COM REMOÇÃO DE OUTLIERS
sih_dengue2 = sih_dengue[(sih_dengue['IDADE'] <= max_values.loc['IDADE', 'value']) 
                        & (sih_dengue['DIAS_PERM'] <= max_values.loc['DIAS_PERM', 'value'])]

In [ ]:
#GERANDO BOXPLOT DE DIAS DE PERMANÊNCIA
sns.set(rc={'figure.figsize':(14, 7)})
sns.set_theme(style="whitegrid", palette="viridis")

box1 = sns.boxplot(x = "DIAS_PERM", data = sih_dengue2)
box1.set(xlabel ="Valores", ylabel = "", title ='Boxplot dias de permanência')

In [ ]:
#GERANDO BOXPLOT DE IDADE
sns.set(rc={'figure.figsize':(14, 7)})
sns.set_theme(style="whitegrid", palette="viridis")

box2 = sns.boxplot(x = "IDADE", data = sih_dengue2)
box2.set(xlabel ="Valores", ylabel = "", title ='Boxplot idade')

In [ ]:
#GERANDO BOXPLOT DE DIAS DE PERMANÊNCIA EM UTI
sns.set(rc={'figure.figsize':(14, 7)})
sns.set_theme(style="whitegrid", palette="viridis")

box3 = sns.boxplot(x = "UTI_MES_TO", data = sih_dengue2)
box3.set(xlabel ="Valores", ylabel = "", title ='Boxplot dias de permanência em UTI')

## 03 - Análise de correlação e associação das variáveis

### 03.1 - Correlação entre variáveis numéricas

In [ ]:
#CALCULANDO CORRELAÇÃO DE PEARSON ENTRE IDADE E DIAS DE PERMANENCIA
corr_idade_diasperm = pearsonr(sih_dengue2['IDADE'], sih_dengue2['DIAS_PERM'])
corr_idade_diasperm

In [ ]:
#CALCULANDO CORRELAÇÃO DE PEARSON ENTRE A PERMANENCIA EM UTI E A PERMANENCIA EM INTERNAÇÃO
corr_diasuti_diasperm = pearsonr(sih_dengue2['UTI_MES_TO'], sih_dengue2['DIAS_PERM'])
corr_diasuti_diasperm

In [ ]:
#CALCULANDO CORRELAÇÃO DE PEARSON ENTRE IDADE E PERMANENCIA EM UTI
corr_diasuti_idade = pearsonr(sih_dengue2['UTI_MES_TO'], sih_dengue2['IDADE'])
corr_diasuti_idade

### 03.2 - Associação entre variáveis categóricas

In [ ]:
#ASSOCIAÇÃO ENTRE MORTE E SEXO
sexo2 = pd.crosstab(sih_dengue2['SEXO'], sih_dengue2['MORTE'])

stat, p, dof, expected = chi2_contingency(sexo2)

prob = 0.95

critical = chi2.ppf(prob, dof)

if abs(stat) >= critical:
	print('Rejeitar H0')
else:
	print('Não se pode rejeitar H0')

alpha = 1.0 - prob

print('significancia=%.3f, p=%.3f' % (alpha, p))

if p <= alpha:
	print('Rejeitar H0')
else:
	print('Não se pode rejeitar H0')

In [ ]:
#ASSOCIAÇÃO ENTRE MORTE E REGIÃO
regiao2 = pd.crosstab(sih_dengue2['MORTE'], sih_dengue2['REGIAO'])

stat, p, dof, expected = chi2_contingency(regiao2)

prob = 0.95

critical = chi2.ppf(prob, dof)

if abs(stat) >= critical:
	print('Rejeitar H0')
else:
	print('Não se pode rejeitar H0')

alpha = 1.0 - prob

print('significancia=%.3f, p=%.3f' % (alpha, p))

if p <= alpha:
	print('Rejeitar H0')
else:
	print('Não se pode rejeitar H0')

In [ ]:
#ASSOCIAÇÃO ENTRE MORTE E UTI
uti2 = pd.crosstab(sih_dengue2['MORTE'], sih_dengue2['MARCA_UTI'])

stat, p, dof, expected = chi2_contingency(uti2)

prob = 0.95

critical = chi2.ppf(prob, dof)

if abs(stat) >= critical:
	print('Rejeitar H0')
else:
	print('Não se pode rejeitar H0')

alpha = 1.0 - prob

print('significancia=%.3f, p=%.3f' % (alpha, p))

if p <= alpha:
	print('Rejeitar H0')
else:
	print('Não se pode rejeitar H0')

## 04 - Treinamento do modelo

### 04.1 - Separando dados em treino e teste

In [ ]:
#SEPARANDO OS DADOS
train, test = train_test_split(sih_dengue2, train_size=0.70, random_state=42)

### 04.2 - Estimando o modelo com todas as variáveis selecionadas

In [ ]:
#ESTIMANDO O MODELO
log_reg = smf.logit("MORTE ~ IDADE + SEXO + DIAS_PERM + MARCA_UTI + UTI_MES_TO + REGIAO", data = train).fit()
print(log_reg.summary())

### 04.3 - Estimando o modelo com todas as variáveis significativas

In [ ]:
#ESTIMANDO O MODELO COM A EXCLUSÃO DA VARIÁVEL NÃO SIGNIFICATIVA
log_reg = smf.logit("MORTE ~ IDADE + SEXO + DIAS_PERM + MARCA_UTI + UTI_MES_TO", data = train).fit()
print(log_reg.summary())

### 04.4 - Gerando predições com os dados de teste

In [ ]:
#ANALISANDO RESULTADOS DO MODELO
test_featues = test.drop("MORTE", axis= 1)

ytest = test["MORTE"]

ypred = log_reg.predict(test_featues)

### 04.5 - Analisando métricas do modelo

In [ ]:
#TABELA DE CONTINGÊNCIA PARA OS RESULTADOS - COM THRESHOLD = 0.5
ypred2 = np.where(ypred >= 0.5, 1, 0)

pd.crosstab(ytest, ypred2, margins= True)

In [ ]:
#CALCULANDO MÉTRICAS DO MODELO
print(classification_report(ytest, ypred2))

In [ ]:
#INCLUINDO DADOS DE RESULTADO DA PREDIÇÃO NO DATAFRAME DE TESTE
test["PRED"] = ypred

## 05 - Alterando o threshold do modelo

### 05.1 - Gerando curva ROC

In [ ]:
#CALCULANDO MÉTRICAS PARA A CURVA ROC
fpr, tpr, thresholds = roc_curve(test["MORTE"], test["PRED"])
roc_auc = auc(fpr, tpr)

print("Área abaixo da curva ROC: %f" % roc_auc)

In [ ]:
#PLOTANDO CURVA ROC
i = np.arange(len(tpr)) 
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})
roc.iloc[(roc.tf-0).abs().argsort()[:1]]

sns.set(rc={'figure.figsize':(20, 10)})
sns.set_theme(style="whitegrid", palette="viridis")

pl.plot(roc['tpr'])
pl.plot(roc['1-fpr'])
pl.xlabel('1-Taxa de Falso Positivo')
pl.ylabel('Taxa de Verdadeiros positivos')

### 05.2 - Gerando melhor threshold que maximize as classificações do modelo

In [ ]:
#DEFININDO MELHOR CUTOFF PARA O MODELO
def optimal_cutoff(target, predicted):

    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.iloc[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 


threshold = optimal_cutoff(test["MORTE"], test["PRED"])
print(threshold)

t = threshold[0]

In [ ]:
#INCLUINDO RESULTADOS NO DATAFRAME
test["PRED_CLASS"] = np.where(test["PRED"] >= t, 1, 0)

### 05.3 - Analisando modelo com novo threshold

In [ ]:
#MATRIZ DE CONFUSÃO
pd.crosstab(test["MORTE"], test["PRED_CLASS"], margins=True)

In [ ]:
#CALCULANDO MÉTRICAS DO MODELO
print(classification_report(test["MORTE"], test["PRED_CLASS"]))

## 06 - Trabalhando o desbalanceamento da base

In [ ]:
#IMPORTANDO PACOTE
from imblearn.over_sampling import SMOTE

In [ ]:
#VISUALISANDO A QUANTIDADE DE VALORES NA TARGET
sih_dengue2['MORTE'].value_counts()

In [ ]:
#SEPARANDO OS DADOS
X_train, y_train = train.drop("MORTE", axis=1), train["MORTE"]

In [ ]:
#GERANDO UMA NOVA AMOSTRA BALANCEADA
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)

train_smote = pd.DataFrame(X_res)
train_smote["MORTE"] = y_res

In [ ]:
#ESTIMANDO O MODELO COM A EXCLUSÃO DA VARIÁVEL NÃO SIGNIFICATIVA
log_reg_smote = smf.logit("MORTE ~ IDADE + SEXO + DIAS_PERM + MARCA_UTI + UTI_MES_TO", data = train_smote).fit()
print(log_reg_smote.summary())

In [ ]:
#ANALISANDO RESULTADOS DO MODELO
test_featues = test.drop("MORTE", axis= 1)
ytest = test["MORTE"]

ypred_smote = log_reg_smote.predict(test_featues)


In [ ]:
#TABELA DE CONTINGÊNCIA PARA OS RESULTADOS - COM THRESHOLD = 0.5
ypred_smote2 = np.where(ypred_smote >= 0.5, 1, 0)

pd.crosstab(ytest, ypred_smote2, margins= True)

In [ ]:
#CALCULANDO MÉTRICAS DO MODELO
print(classification_report(ytest, ypred_smote2))